# Backcasting Test Notebook

_Loren Champlin_

Adapted from _Adarsh Pyarelal_'s WM 12 Month Evaluation Notebook 

As always, we begin with imports, and print out the commit hash for a rendered
version of the notebook.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pickle
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
from delphi.visualization import visualize
import delphi.jupyter_tools as jt
import numpy as np
import pandas as pd
from scipy import stats
#Comment out the next line if you do not have the delphi.db file. 
from delphi.db import engine
jt.print_commit_hash_message()
import random as rm
import delphi.evaluation as EN
import delphi.AnalysisGraph as AG
import warnings
#warnings.filterwarnings("ignore")
import logging
logging.getLogger().setLevel(logging.CRITICAL)
from indra.statements import (
    Concept,
    Influence,
    Evidence,
    Event,
    QualitativeDelta,
)
from delphi.utils.indra import *
from delphi.utils.shell import cd
import seaborn as sns
import matplotlib.pyplot as plt
from delphi.utils.fp import flatMap, take, ltake, lmap, pairwise, iterate, exists
import time

Here I will set random seeds

In [ ]:
np.random.seed(87)
rm.seed(87)

In [ ]:
concepts = {
    "conflict": {
        "grounding": "UN/events/human/conflict",
        "delta": {"polarity": 1, "adjective": ["small"]},
    },
    "food security": {
        "grounding": "UN/entities/human/food/food_security",
        "delta": {"polarity": -1, "adjective": ["large"]},
    },
    "migration": {
        "grounding": "UN/events/human/human_migration",
        "delta": {"polarity": 1, "adjective": ['small']},
    },
    "product": {
        "grounding": "UN/entities/natural/crop_technology/product",
        "delta": {"polarity": 1, "adjective": ['large']},
    },
    "economic crisis": {
        "grounding": "UN/events/human/economic_crisis",
        "delta": {"polarity": 1, "adjective": ["large"]},
    },
    "precipitation": {
        "grounding": "UN/events/weather/precipitation",
        "delta": {"polarity": 1, "adjective": []},
    },
    "inflation": {
        "grounding": "UN/entities/human/financial/economic/inflation",
        "delta": {"polarity": -1, "adjective": ["large"]},
    },

}

def make_event(concept, attrs):
    return Event(
        Concept(
            attrs["grounding"],
            db_refs={"TEXT": concept, "UN": [(attrs["grounding"], 0.8)]},
        ),
        delta=QualitativeDelta(
            attrs["delta"]["polarity"], attrs["delta"]["adjective"]
        ),
    )


def make_statement(event1, event2):
    return Influence(
        event1,
        event2,
        evidence=Evidence(
            annotations={
                "subj_adjectives": event1.delta.adjectives,
                "obj_adjectives": event2.delta.adjectives,
            }
        ),
    )

events = {
    concept: make_event(concept, attrs) for concept, attrs in concepts.items()
}

s1 = make_statement(events["conflict"], events["food security"])
s2 = make_statement(events["migration"], events["product"])
s3 = make_statement(events["migration"], events["economic crisis"])
s4 = make_statement(events["precipitation"], events["inflation"])
s5 = make_statement(events["inflation"],events["migration"])

Now we load the Causal Analysis Graph (CAG). This is CAG was inferred by reading in a JSON corpus and was pruned and adjusted to be human migration centered. Also is a list of the nodes contained in the CAG

In [ ]:
start_time = time.time()
G = AG.AnalysisGraph.from_statements(get_valid_statements_for_modeling([s5]))


In [ ]:
G.map_concepts_to_indicators()
G.set_indicator("UN/events/human/human_migration","New asylum seeking applicants","UNHCR")

In [ ]:
EN.train_model(G,2015,1,2015,12,10000,90000)

In [ ]:
EN.generate_predictions(G,2016,1,2016,12)
end_time = time.time()
total_time = end_time-start_time



In [ ]:
total_time

In [ ]:
EN.pred_to_array(G,"New asylum seeking applicants")[:,0].mean()

In [ ]:
EN.mean_pred_to_df(G,'New asylum seeking applicants',true_vals=True)

In [ ]:
EN.pred_plot(G,'New asylum seeking applicants',plot_type='Comparison',save_as=None)

In [ ]:
n = 2
query = " ".join(
        [
            "select Source, Indicator from concept_to_indicator_mapping",
            "where `Concept` like 'UN/entities/human/financial/economic/inflation'",
        ]
    )

results = engine.execute(query)

pd.DataFrame(results, columns=results.keys())

Next we map indicator variables to nodes. For the most part indicator variables can be inferred from available data and texts, but we can also manually map indicators to nodes. There is also a list of the indicator variables in the same order as the list of nodes above (i.e "Claims on other sectors of the domestic economy" is attached to "UN/events/human/economic_crisis".

In [ ]:
query = " ".join(
        [
            f"select * from indicator",
            f"where `Variable` like 'New asylum seeking applicants' and `Year` is '2015'",
        ]
    )

results = engine.execute(query)

df = pd.DataFrame(results,columns=results.keys())
df

In [ ]:
x = 5
y = x

y = 2

print(x)